O código abaixo não pode ser executado em um notebook Colab ou Jupyter, devido à incompatibilidade com a API do GitHub.

Execute o código, conforme as instruções documentadas, em outro ambiente de execução da linguagem R.


Instala pacote API e outros pacotes necessários:

In [ ]:
install.packages("remotes")
install.packages("readxl")
remotes::install_github("jjesusfilho/tjsp", force=TRUE)
remotes::install_github("leeper/rio", force=TRUE)

Carregar planilha de termos:

In [ ]:
#a planilha "termos_utilizados.xlsx" deve estar upada no ambiente de execucao, no diretorio padrao
library(readxl)
p <- read_excel("termos_utilizados.xlsx")
ter <- p$termo #cria vetor de termos

Função para tratar existência e criação dos diretórios:

In [ ]:
verificaDiretorio = function(t) {
  if (dir.exists(t)) {
    unlink(t) # Se o diretório já existe, exclui-o
  }
  dir.create(t) # Cria um novo diretório
}

Função para baixar as páginas HTML referentes aos resultados totais encontrados:

In [ ]:
baixaPaginas = function(t) {
    library(tjsp)
    busca <- paste('"', t, '"', sep = "") #constroi string de busca
    autenticar(login="xxxxx", password="xxxxx")
    #busca no portal
    print(t)
    out <- tryCatch(
    {
    tjsp_baixar_cjsg(livre=busca, diretorio=t)
    },
    error=function(cond) {
      message("download nao funcionou - ERRO - baixando paginas novamente....")
      message(cond)
      baixaPaginas(t)
      # Choose a return value in case of error
      return(NA)
    },
      warning=function(cond) {
      message("download nao funcionou - warning - baixando paginas novamente....")
      message(cond)
      baixaPaginas(t)
      # Choose a return value in case of error
      return(NA)
    }
  )
}

In [ ]:
verificaPaginas = function(t) {

  arquivos <- list.files(t, full.names = TRUE)
  if(length(arquivos)>1){
    for (arquivo in arquivos) {
      tamanho <- file.info(arquivo)$size
      if (tamanho==0) {
        unlink(t)
        dir.create(t)
        baixaPaginas(t)
        verificaPaginas(t)
        break
      }
    }
  }
}

Função para criar o dataframe de resultados:

In [ ]:
criaDataFrame = function(tabela,vetor_acordaos,textos){
  df <- data.frame(
    classe = tabela$classe,
    assunto = tabela$assunto,
    relator = tabela$relator,
    comarca = tabela$comarca,
    orgao_julgador = tabela$orgao_julgador,
    data_julgamento = tabela$data_julgamento,
    data_publicacao = tabela$data_publicacao,
    processo = tabela$processo,
    idacordao = vetor_acordaos,
    acordao = textos
  )
  return(df)
}

Função para leitura de um acórdão individual e atualização dos resultados:

In [ ]:
leituraAcordaos = function(t,i,j){
    library(tjsp)
    nome <- paste(t,"/","cdacordao_", i, ".pdf", sep = "") #armazena nome do arquivo baixado, com diretorio

    while(file.info(nome)$size == 0){
      file.remove(nome)
      tjsp_baixar_acordaos_cjsg(i, diretorio = t) #baixa arquivo referente ao codigo do acordao
    }

    leitura <- paste(" ", (ler_acordaos(arquivos = nome,remover_assinatura = TRUE,combinar = TRUE))$julgado, sep="") #armazena apenas a leitura do texto do acordao

  return(leitura)
}

Função para buscar todos os acórdãos referentes aos processos encontrados para um determinado termo:

In [ ]:
buscaAcordaos = function(tabela,t){
  vetor_acordaos_char <- tabela$cdacordao
  vetor_acordaos <- as.integer(vetor_acordaos_char)
  textos <- c()

  j<-1
  for(i in vetor_acordaos){ #percorre os resultados, a fim de baixar os respectivos acordaos
      library(tjsp)
      autenticar(login="xxxx", password="xxxx") #autenticacao no tjsp
      tjsp_baixar_acordaos_cjsg(i, diretorio = t) #baixa arquivo referente ao codigo do acordao
        tryCatch({
          textos <- c(textos,leituraAcordaos(t,i,j))
        }, error = function(e) {
          textos <- c(textos,"erro na leitura do acordao")
        })
       #le arquivo e atualiza resultados
      j<-j+1
  }
  df <- criaDataFrame(tabela,vetor_acordaos,textos)
  return(df)
}

Laço total de busca e tratamento de todos os termos:

In [ ]:
processa_termo = function(t) {
    library(tjsp)
    library("rio")
    nome_final <- paste("com-acordaos-", t, ".csv", sep = "")

    if (!file.exists(nome_final)) {
        verificaDiretorio(t)
        busca <- paste('"', t, '"', sep = "") #constroi string de busca
        tjsp_baixar_cjsg(livre=busca, diretorio=t)
        Sys.sleep(30)
        tabela <- tjsp_ler_cjsg(diretorio = t)
        vetor_processos <- tabela$processo

        if (length(vetor_processos) > 0) {
            df <- buscaAcordaos(tabela, t)
            export(df, nome_final)
        } else {
            unlink(t, recursive = TRUE)
        }
    }
}

Comandos principais:

In [ ]:
library(tjsp)
#autenticacao no tjsp

for(t in ter) {
  autenticar(login="xxxx", password="xxxxx")
  tryCatch({
    processa_termo(t)
  }, error = function(e) {
    cat("termo não funcionou\n")
  })
}